In [1]:
import tensorflow as tf
import numpy as np
import time

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
from IcGanLayer import gan_loss, classify_loss, generator

In [2]:
n_classes = 20
batch_size = 64
n_epochs

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement=True

In [3]:
with tf.device('/device:GPU:0'):
    x = tf.placeholder(tf.float32, shape=[None, 64, 64,  3], name='x')
    y = tf.placeholder(tf.float32, shape=[None, n_classes], name='y')
    
    loss_pred, loss_gen, gen = gan_loss(x, y, n_classes)
    loss_cls, Ey = classify_loss(x, y, n_classes)
    
    saver = tf.saver()
    evaluate = tf.equal(tf.round(Ey), y)
    accuracy = tf.reduce_mean(tf.cast(evaluate),dtype=tf.float32)
    
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        pred_optimizer = tf.train.AdamOptimizer(0.0002, beta1=0.5).minimize(loss_pred)
        gen_optimizer = tf.train.AdamOptimizer(0.0002, beta1=0.5).minimize(loss_gen)
        cls_optimizer = tf.train.AdamOptimizer(0.0002, beta1=0.5).minimize(loss_cls)
    
    

In [ ]:
with tf.Session(config=config) as sess:
    for epoch in range(n_epochs):
        pred_cost, gen_cost = sess.run([pred_optimizer, gen_optimizer], feed_dict={x:dataset.nextbatch(), y:label.nextbatch()})
        print('discriminator loss: %f, generator loss: %f' % (pred_cost, gen_cost))
        
    test_image = sess.run([gen], feed_dict={x:test_dataset.nextbatch(), y:test_label.nextbatch()})
    show_image(test_image)
    saver.save(sess, 'save/to/path')

In [ ]:
with tf.Session(config=config) as sess:
    for epoch in range(n_epochs):
        cls_cost = sess,run([cls_optimizer], feed_dict={x:dataset.nextbatch(), y:label.nextbatch()})
        print('classifer loss: %f, accuracy: %f' % (cls_cost, accuracy.eval()))
    saver.save(sess, 'save/to/path')
        